In [9]:
import pyspark.sql.functions as F
import time

StatementMeta(, 57171b50-bd83-4d6c-bfc3-391cfed22981, 11, Finished, Available, Finished)

In [10]:
%run NB_Credentials

StatementMeta(, 57171b50-bd83-4d6c-bfc3-391cfed22981, 12, Finished, Available, Finished)

In [11]:
# Iniciar tiempo de ejecución
start_time = time.time()

StatementMeta(, 57171b50-bd83-4d6c-bfc3-391cfed22981, 13, Finished, Available, Finished)

In [12]:
# Definir nombres de las tablas
BRONZE_FACT_POST = "bronze_fact_post"
BRONZE_DIM_USERS = "bronze_dim_users"
BRONZE_DIM_TAGS = "bronze_dim_tags"

SILVER_FACT_POST = "silver_fact_post"
SILVER_DIM_USERS = "silver_dim_users"
SILVER_DIM_TAGS = "silver_dim_tags"

StatementMeta(, 57171b50-bd83-4d6c-bfc3-391cfed22981, 14, Finished, Available, Finished)

In [13]:
# Cargar la tabla fact_post desde Bronze
df_fact_post = spark.read.format("delta").load(f"{BRONZE_PATH}/{BRONZE_FACT_POST}")

# Transformaciones
df_fact_post = (df_fact_post
    .withColumn("date", F.to_date("date"))  # Convertir la columna date a formato fecha
    .filter(F.col("post").isNotNull())  # Eliminar posts vacíos
    .dropDuplicates(["author", "post", "date"])  # Evitar registros duplicados
)

# Mostrar datos transformados
# display(df_fact_post)

# Guardar en Silver
df_fact_post.write.format("delta").mode("overwrite").save(f"{SILVER_PATH}/{SILVER_FACT_POST}")

StatementMeta(, 57171b50-bd83-4d6c-bfc3-391cfed22981, 15, Finished, Available, Finished)

In [14]:
# Cargar la tabla dim_users desde Bronze
df_dim_users = spark.read.format("delta").load(f"{BRONZE_PATH}/{BRONZE_DIM_USERS}")

# Transformaciones
df_dim_users = (df_dim_users
    .dropDuplicates(["author"])
    .fillna({
        "display_name": "Unknown",
        "description": "No description available",
        "followers_count": 0,
        "follows_count": 0,
        "posts_count": 0
    })
    .withColumn("followers_count", F.col("followers_count").cast("int")).where(F.col("followers_count") >= 0)
    .withColumn("follows_count", F.col("follows_count").cast("int")).where(F.col("follows_count") >= 0)
    .withColumn("posts_count", F.col("posts_count").cast("int"))
)
# Mostrar datos transformados
# display(df_dim_users)

# Guardar en Silver
df_dim_users.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save(f"{SILVER_PATH}/{SILVER_DIM_USERS}")

StatementMeta(, 57171b50-bd83-4d6c-bfc3-391cfed22981, 16, Finished, Available, Finished)

In [15]:
# Cargar la tabla dim_tags desde Bronze
df_dim_tags = spark.read.format("delta").load(f"{BRONZE_PATH}/{BRONZE_DIM_TAGS}")

# Transformaciones
df_dim_tags = (df_dim_tags
    .withColumn("date", F.to_date("date"))
    .filter(F.col("hashtag").isNotNull())
    .dropDuplicates(["author", "hashtag", "date"])
    .withColumn("hashtag", F.lower(F.col("hashtag")))  # Convertir hashtags a minúsculas
    .filter(F.length(F.col("hashtag")) > 1)  # Filtrar hashtags demasiado cortos
)

# Mostrar datos transformados
# display(df_dim_tags)

# Guardar en Silver
df_dim_tags.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save(f"{SILVER_PATH}/{SILVER_DIM_TAGS}")


StatementMeta(, 57171b50-bd83-4d6c-bfc3-391cfed22981, 17, Finished, Available, Finished)

In [16]:
# Calcular tiempo de ejecución
end_time = time.time()
execution_time = end_time - start_time
print(f"\n⏳ Tiempo total de ejecución: {execution_time:.2f} segundos")

StatementMeta(, 57171b50-bd83-4d6c-bfc3-391cfed22981, 18, Finished, Available, Finished)


⏳ Tiempo total de ejecución: 14.38 segundos
